In [131]:
# Code in this cell largely follows that written by the starter code writer.

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [132]:
import json

In [133]:
cities_df = pd.read_csv("../WeatherPy/output_files/city_data.csv")
cities_df.head()

,Unnamed: 0,row,City,Latitude,Longitude,Temperature (degrees F),Humidity,Clouds (%),Winds Speed
0,0,0,ous,60.91,61.52,56.08,67,28,1.43
1,1,1,rikitea,-23.12,-134.97,70.47,81,88,11.88
2,2,2,bredasdorp,-34.53,20.04,44.60,87,49,6.93
3,3,3,bluff,-46.60,168.33,39.99,96,100,5.48
4,4,4,busselton,-33.65,115.33,44.42,83,21,9.33


In [134]:
gmaps.configure(api_key=g_key)

In [135]:
# Code in this cell pulled from class activity Evr_Jupyter_Gmaps

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [136]:
# Code here is largely pulled form class activity Stu_BankDeserts_Heatmap

locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df["Humidity"].astype(float)


# I tried setting max_intensity to 100, which is the highest humidity,
# but I like the way it looks with max_intensity 250 and point_radius 8.
# It is smoother.

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=250,
                                 point_radius = 8)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [137]:
# Narrowing down the list according to desired vacation conditions.

low_temp_bool = cities_df["Temperature (degrees F)"] >= 50 
high_temp_bool = cities_df["Temperature (degrees F)"] <= 90

high_hum_bool = cities_df["Humidity"] <= 50

high_clouds_bool = cities_df["Clouds (%)"] <= 50

high_winds_bool = cities_df["Winds Speed"] < 10
low_winds_bool = cities_df["Winds Speed"] > 4

# Want to go to the Southern Hemisphere to see the Large Magellanic Cloud
high_lat_bool = cities_df["Latitude"] < 0


vacation_df = cities_df.loc[low_temp_bool & high_temp_bool & high_hum_bool & high_clouds_bool \
                            & high_winds_bool & low_winds_bool & high_lat_bool, :]


vacation_df


,Unnamed: 0,row,City,Latitude,Longitude,Temperature (degrees F),Humidity,Clouds (%),Winds Speed
91,91,104,bela vista,-22.11,-56.52,89.17,19,0,5.91
106,106,123,kang,-23.68,22.79,57.31,34,0,4.29
196,196,224,morro agudo,-20.73,-48.06,84.20,20,0,5.82
284,284,318,monte carmelo,-18.72,-47.50,74.01,33,0,8.34
323,323,360,moron,-34.65,-58.62,78.80,44,47,5.99
343,343,382,senanga,-16.12,23.27,64.04,28,0,4.63
360,360,400,outjo,-20.12,16.15,52.81,43,0,6.17
393,393,435,jaciara,-15.97,-54.97,89.94,21,0,8.16
429,429,474,unai,-16.36,-46.91,78.89,29,0,8.75
462,462,510,viedma,-40.81,-63.00,63.00,42,0,8.99


In [153]:
target_search = ""
target_radius = 5000
target_type = "hotel"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


hotel_list = []
coord_list = []

for row in vacation_df.index:
    

    # geocoordinates
    target_coordinates = str(vacation_df.loc[row]["Latitude"]) + ", " + str(vacation_df.loc[row]["Longitude"])
        
    coord_list.append((vacation_df.loc[row]["Latitude"], vacation_df.loc[row]["Longitude"])) 

        
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    places_data = response.json()

    hotel = places_data["results"][1]["name"]
    
    hotel_list.append(hotel)
    

print(hotel_list)    
    
    
#     print(places_data["results"][1]["name"])
    
#     #hotel = places_data["results"][1]["name"]
    
#     print(hotel)
    
#     # Print the json (pretty printed)
    
    
    
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    
    
    
#     #print(places_data["results"][1]["name"])
#     # print(places_data["results"][0]["vicinity"])

['Bella Vista Palace Hotel', 'Kang Ultra Stop', '2189/MORRO AGUDO', 'Rua Parana 181', 'Solo Deportes', 'Jauza Tech Universal', "Weimann's Garage", 'Jaciara Service', 'Hotel Regente', 'Hotel Cristal']


In [154]:
vacation_df["Hotel"] = hotel_list


c:\users\potas\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [155]:
vacation_df.head(10)


,Unnamed: 0,row,City,Latitude,Longitude,Temperature (degrees F),Humidity,Clouds (%),Winds Speed,Hotel
91,91,104,bela vista,-22.11,-56.52,89.17,19,0,5.91,Bella Vista Palace Hotel
106,106,123,kang,-23.68,22.79,57.31,34,0,4.29,Kang Ultra Stop
196,196,224,morro agudo,-20.73,-48.06,84.20,20,0,5.82,2189/MORRO AGUDO
284,284,318,monte carmelo,-18.72,-47.50,74.01,33,0,8.34,Rua Parana 181
323,323,360,moron,-34.65,-58.62,78.80,44,47,5.99,Solo Deportes
343,343,382,senanga,-16.12,23.27,64.04,28,0,4.63,Jauza Tech Universal
360,360,400,outjo,-20.12,16.15,52.81,43,0,6.17,Weimann's Garage
393,393,435,jaciara,-15.97,-54.97,89.94,21,0,8.16,Jaciara Service
429,429,474,unai,-16.36,-46.91,78.89,29,0,8.75,Hotel Regente
462,462,510,viedma,-40.81,-63.00,63.00,42,0,8.99,Hotel Cristal


In [156]:
coord_list


[(-22.11, -56.52),
 (-23.68, 22.79),
 (-20.73, -48.06),
 (-18.72, -47.5),
 (-34.65, -58.62),
 (-16.12, 23.27),
 (-20.12, 16.15),
 (-15.97, -54.97),
 (-16.36, -46.91),
 (-40.81, -63.0)]

In [157]:
# This cell pulled largely from class activity Evr_Jupyter_Gmaps

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coord_list)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…